In [ ]:
# !pip install kaggle

# !cp kaggle.json ~/.kaggle/

# Small SIDD dataset
# !kaggle datasets download -d rajat95gupta/smartphone-image-denoising-dataset

In [ ]:
# !unzip 'smartphone-image-denoising-dataset.zip'

In [ ]:
# Dataload
# augment
# visualize
# train
# save model
# test

In [ ]:
import argparse
import os
import numpy as np
import math
import itertools
import time
import datetime
import sys



import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

from models import *
from datasets import *

import torch.nn as nn
import torch.nn.functional as F
import torch

import matplotlib.pyplot as plt


In [ ]:
epoch = 0
n_epochs = 200
dataset_name = "SIDD_Small_sRGB_Only"
batch_size = 1
lr = 0.0002
b1 = 0.5
b2 = 0.999
decay_epoch = 100
n_cpu = 8
img_height = 256
img_width = 256
channels = 3
sample_interval = 500
checkpoint_interval = -1

# Create Dataset

In [ ]:
import os 
import cv2
import numpy as np
from tqdm import tqdm
from glob import glob
from sklearn.model_selection import train_test_split

REBUILD_DATA=True

class Denoising():
    IMG_SIZE=256 
    label_dir='SIDD_Small_sRGB_Only/Data/' #dataset directory
    training_data=[]
    validation_data=[]
    
    def __init__(self):
        self.train_images, self.val_images = train_test_split(os.listdir(self.label_dir)[:20], test_size=0.2, random_state=42)

    def make_training_and_validation_data(self):
        print(self.train_images)
        print(self.val_images)

        
        for image_folder in tqdm(os.listdir(self.label_dir)[:20]):
            # print(image_folder)
            for img_path in glob(self.label_dir + image_folder + '/*'):
                try:
                    img=cv2.imread(img_path)
                    img=cv2.resize(img,(self.IMG_SIZE,self.IMG_SIZE))
                    
                    # Check if the image belongs to train or validation set
                    if image_folder in self.train_images:
                        self.training_data.append(np.array(img))
                    elif image_folder in self.val_images:
                        self.validation_data.append(np.array(img))

                except Exception as e:
                    pass
            
        np.save('SIDD_Small_sRGB_Only_training_data.npy',self.training_data)
        np.save('SIDD_Small_sRGB_Only_validation_data.npy',self.validation_data)
        
# if (REBUILD_DATA):
#     denoising=Denoising()
#     denoising.make_training_and_validation_data()


# Load Dataset

In [ ]:
#loading pickle file
training_data=np.load('SIDD_Small_sRGB_Only_training_data.npy',allow_pickle=True)
val_data=np.load('SIDD_Small_sRGB_Only_validation_data.npy',allow_pickle=True)
len(training_data), len(val_data)

# Visualize Data

In [ ]:
import matplotlib.pyplot as plt

def display_images(data, vis_num):
    num_cols = 2
    num_rows = vis_num
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 10))
    for i in range(num_rows):
        for j in range(num_cols):
            index = i * num_cols + j
            axs[i, j].imshow(data[index])
            axs[i, j].set_title('Original' if index % 2 == 0 else 'Noisy')
            axs[i, j].axis('off')
    plt.tight_layout()
    plt.show()


# Assuming you have already loaded training_data and val_data
vis_num = 0
display_images(training_data, vis_num)
display_images(val_data, vis_num)

In [1]:
!python pix2pix.py --train_dataset_loc 'SIDD_Small_sRGB_Only_training_data.npy' --test_dataset_loc 'SIDD_Small_sRGB_Only_validation_data.npy' --n_epochs 1

## Pix2Pix

In [ ]:
cuda = True if torch.cuda.is_available() else False

# Loss functions
criterion_GAN = torch.nn.MSELoss()
criterion_pixelwise = torch.nn.L1Loss()

# Loss weight of L1 pixel-wise loss between translated image and real image
lambda_pixel = 100

# Calculate output of image discriminator (PatchGAN)
patch = (1, img_height // 2 ** 4, img_width // 2 ** 4)

# Initialize generator and discriminator
generator = GeneratorUNet()
discriminator = Discriminator()

if cuda:
    generator = generator.cuda()
    discriminator = discriminator.cuda()
    criterion_GAN.cuda()
    criterion_pixelwise.cuda()

if epoch != 0:
    # Load pretrained models
    generator.load_state_dict(torch.load("saved_models/%s/generator_%d.pth" % (dataset_name, epoch)))
    discriminator.load_state_dict(torch.load("saved_models/%s/discriminator_%d.pth" % (dataset_name, epoch)))
else:
    # Initialize weights
    generator.apply(weights_init_normal)
    discriminator.apply(weights_init_normal)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1,b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))



from DataLoader import CustomDataset
from sklearn.model_selection import train_test_split

# Define transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the image
    transforms.ToTensor(),           # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Set root directory
# root_dir = 'data'
root_dir='SIDD_Small_sRGB_Only/Data/' #dataset directory


# Create dataset
dataset = CustomDataset(root_dir, transform=transform)

# Split dataset into train and test sets
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Create dataloaders for train and test sets
dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)



# Tensor type
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
real_A = Variable(batch[0].type(Tensor))

In [ ]:
real_A.shape

In [ ]:
fake_B = generator(real_A)

In [ ]:
n_epochs

In [ ]:
prev_time = time.time()

for epoch in range(epoch, 2 ):
    for i, batch in enumerate(dataloader):

        # Model inputs
        real_A = Variable(batch[0].type(Tensor))
        real_B = Variable(batch[1].type(Tensor))

        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((real_A.size(0), *patch))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((real_A.size(0), *patch))), requires_grad=False)

        # ------------------
        #  Train Generators
        # ------------------

        optimizer_G.zero_grad()

        # GAN loss
        fake_B = generator(real_A)
        pred_fake = discriminator(fake_B, real_A)
        loss_GAN = criterion_GAN(pred_fake, valid)
        # Pixel-wise loss
        loss_pixel = criterion_pixelwise(fake_B, real_B)

        # Total loss
        loss_G = loss_GAN + lambda_pixel * loss_pixel

        loss_G.backward()

        optimizer_G.step()

In [ ]:
# python pix2pix.py --train_dataset_loc 'SIDD_Small_sRGB_Only_training_data.npy' --test_dataset_loc 'SIDD_Small_sRGB_Only_validation_data.npy' --n_epochs 1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
import os



# Define the generator model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Define the discriminator model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(6, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

# Define loss functions
criterion_GAN = nn.BCELoss()
criterion_L1 = nn.L1Loss()

# Initialize optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to the device
generator.to(device)
discriminator.to(device)

# Define training parameters
num_epochs = 100
batch_size = 8
lr = 0.0002
size = 256

# Define dataset and dataloader
transform = transforms.Compose([
    transforms.Resize(size),
    transforms.CenterCrop(size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# dataset = ImageFolder(root='dataset_path', transform=transform)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    print(epoch)
    for i, data in enumerate(dataloader):
        print(i)
        real_images, _ = data
        real_images = real_images.to(device)
        
        # Train generator
        optimizer_G.zero_grad()
        
        # Generate fake images
        fake_images = generator(real_images)
        
        # Train discriminator with real images
        real_validity = discriminator(torch.cat((real_images, real_images), 1))
        real_labels = torch.ones(real_validity.size()).to(device)
        real_loss = criterion_GAN(real_validity, real_labels)
        
        # Train discriminator with fake images
        fake_validity = discriminator(torch.cat((real_images, fake_images.detach()), 1))
        fake_labels = torch.zeros(fake_validity.size()).to(device)
        fake_loss = criterion_GAN(fake_validity, fake_labels)
        
        # Total discriminator loss
        discriminator_loss = (real_loss + fake_loss) / 2
        
        discriminator_loss.backward()
        optimizer_D.step()
        
        # Train generator
        optimizer_G.zero_grad()
        
        # Adversarial loss
        validity = discriminator(torch.cat((real_images, fake_images), 1))
        target_labels = torch.ones(validity.size()).to(device)
        adversarial_loss = criterion_GAN(validity, target_labels)
        
        # Pixel-wise loss
        pixel_loss = criterion_L1(fake_images, real_images)
        
        # Total generator loss
        generator_loss = adversarial_loss + 100 * pixel_loss
        
        generator_loss.backward()
        optimizer_G.step()
        
        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch+1, num_epochs, i+1, len(dataloader), discriminator_loss.item(), generator_loss.item())
        )
        
        if i % 100 == 0:
            save_image(fake_images, "images/%d_%d.png" % (epoch+1, i+1), normalize=True)


In [ ]:
discriminator(torch.cat((real_images, fake_images.detach()), 1))

In [ ]:
torch.cat((real_images, fake_images.detach()),1)